## ASTER Version 4 Northup 

This notebook provides a protoype function to convert the Level-2 V4 ASTER Products from their archieved rotated map orientation to a northup transformation.

The goals include:
1. Convert Rotated ASTER Data to Northup to support mosaicing 
2. Preserve input paramters (height, wideth, crs, pixel centers) in order to preserve alignment between northup 
3. Write transformed tifs as some output that can be viewed in a GIS to compare with input data

As of 12/9/25 this method was tested on AST_07 for one acquisition all bands. Still need to try with the other Level-2 Products (Visable: AST_09, AST_09X, AST_07X) and (Thermal:  AST_09T, AST_08, AST_05).

Ideally I think we whould want to create and xarray dataset for containing each band. Started that with section 2 the merged xarray dataset appears to have been created but the .nc outputs look very strange in a GIS. Still work in progress.



In [28]:
import rioxarray as rxr
import xarray as xr
import rasterio as rio
from rasterio import transform
from osgeo import gdal
from rasterio.warp import Resampling
import os
import numpy as np
from rasterio.transform import from_origin
from pyproj import Transformer
import math
import earthaccess

In [29]:
earthaccess.login(persist=True)

In [30]:
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
gdal.SetConfigOption('GDAL_HTTP_MAX_RETRY', '10')
gdal.SetConfigOption('GDAL_HTTP_RETRY_DELAY', '0.5')

In [31]:
url = [' https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B04.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B05.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B06.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B07.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B08.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B09.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_B01.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_B02.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_B03N.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_QA_DataPlane.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_QA_DataPlane2.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane2.tif'

]

### Working Draft Northup Transformation Function

In [37]:
""" 
Basic Workflow:

1. Work with rotated TIF and load into xarray
    a. Load rotated tif as xarray data array with rioxarray
    b. Establish no data value the input products do not have this, these tifs have some boarder
       with a datavalue of zero, trying to handling this for future mosaic
    c. Setting dtype as int16 because in previous itterations for some reason rioxarray was setting
       a dtype of int64 which gdal did not reconsize resulting in empty geotiffs on the output
    d. Get size columns and rows of rotated xarray
    e. Get rotated transformation of rotated xarray
    f. Calculate Corner Coordinates for rotated tif

2. Create northup transformed xarray data arrary for each tif
   a. Calculate Pixel Size for Northup 
   b. Calculate shape (width and height for northup transformed xarray)
   c. Create northup transformation
   d. Create northup transformed data xarray array with same pixel center and shape shape as rotated xarray data array
"""

def rotated_to_north_up(input_file, 
                        nodata = 0,
                        resampling=Resampling.nearest):
    
    rotated = rxr.open_rasterio(input_file, masked = True)
    rotated = rotated.rio.write_nodata(nodata)
    rotated = rotated.astype('int16')

    h = rotated.sizes['y']
    w = rotated.sizes['x']
    transform = rotated.rio.transform()

    coords = np.array([
    transform * (0.5, 0.5),
    transform * (w - 0.5 ,0.5),
    transform * (0.5 ,h - 0.5),
    transform * (w - 0.5, h - 0.5),
    ])

    xs = coords[:, 0]
    ys = coords[:, 1]

    xmin, xmax = xs.min(), xs.max()
    ymin, ymax = ys.min(), ys.max()

    res_x = np.sqrt(transform.a**2 + transform.b**2)
    res_y = np.sqrt(transform.d**2 + transform.e**2)

    width = int(np.round((xmax - xmin) / res_x))
    height = int(np.round((ymax - ymin) /res_y))

    north_up = from_origin(xmin, ymax, res_x, res_y)

    north = rotated.rio.reproject(rotated.rio.crs, shape = (height, width), transform = north_up, resampling = resampling)

    print(north.min().values,north.max().values)

    return north

### Section 1 Transform Northup and Make a GeoTIFF

In [36]:
for tif in url:
    print('Processing ', tif)
    north = rotated_to_north_up(
        input_file =  tif)
    output_file = tif.split('/')[6][:-4]
    north.rio.to_raster(output_file+'north_up.tif', driver='GTIFF')

Processing   https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B04.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 577
Processing  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B05.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 475
Processing  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B06.tif


...

0 0
Processing  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 144
Processing  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane2.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 0


### Section 2 Transform Northup and Create Xarray Dataset with each transformed tif as a coordinate and export as stacked GeoTIFF or netcdf

In [ ]:
north_up_array = []
for tif in url:
    print('Processing ', tif)
    north = rotated_to_north_up(
        input_file =  tif
    )

    #Deal with adding band number as coordinate
    output_file = tif.split('/')[6][:-4]
    if 'QA' in output_file:
        band_name = '_'.join(output_file.split('_')[5:8])
    else:
        band_name = '_'.join(output_file.split('_')[5:7])

    north = north.squeeze('band', drop = True)
    north = north.rename(band_name)
    north_up_array.append(north)

#make dataset of all
aster_data_set = xr.merge(north_up_array)

#make stacked geotiff
#looks strange in GIS something is wrong
aster_data_set.rio.to_raster('stacked.tif', compress = 'deflate', dtype = str(north.dtype))

#make into netcdf
#looks strange in GIS something is wrong
aster_data_set.to_netcdf('aster.nc')

Processing   https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B04.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 577
Processing  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B05.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 475
Processing  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B06.tif


...

c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 144
Processing  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane2.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


0 0


C:\Users\nroberts\AppData\Local\Temp\1\ipykernel_27752\3864698539.py:20: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'x' ('x',) The recommendation is to set join explicitly for this case.
  aster_data_set = xr.merge(north_up_array)
C:\Users\nroberts\AppData\Local\Temp\1\ipykernel_27752\3864698539.py:20: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'y' ('y',) The recommendation is to set join explicitly for this case.
  aster_data_set = xr.merge(north_up_array)
C:\Users\nroberts\AppData\Local\Temp\1\ipykernel_2